In [ ]:
import os
import numpy as np
import xarray as xr
import hsmm_mvpy as hmp
from mne.io import read_info

cpus = 3 #set the number of cores to use for all the code

epoch_data = xr.load_dataset(os.path.join('../sample_data/sample_data.nc'))

#We create also the position array for the electrodes as we are going to need them to plot the event topologies
positions = read_info(os.path.join('../sample_data/eeg/processed_0023_epo.fif'), verbose=False)

#And we transform as in the previous tutorial
hmp_data = hmp.utils.transform_data(epoch_data, apply_zscore='trial', n_comp=4)

### Applying HsMM separately for the speed condition

Initializing data for the speed condition

In [ ]:
hmp_speed_data = hmp.utils.condition_selection(hmp_data, epoch_data, 'SP', variable='cue')#select the conditions where participants needs to be fast
init_speed = hmp.models.hmp(hmp_speed_data,
                       sfreq=epoch_data.sfreq, cpus=cpus)

In [ ]:
estimates_speed = init_speed.fit()

In [ ]:
hmp.visu.plot_topo_timecourse(epoch_data, estimates_speed, positions, init_speed, 
                          times_to_display = np.mean(init_speed.ends - init_speed.starts))

In [ ]:
speed_epoch = epoch_data.where(epoch_data['cue'].str.contains('SP'), drop=True)
bootstrapped = hmp.resample.bootstrapping(data=speed_epoch, dim=['epochs','participant'], n_iterations=10, 
                                                 init=init_speed, positions=positions, sfreq=epoch_data.sfreq, cpus=cpus)

hmp.visu.plot_bootstrap_results(bootstrapped, positions, init_speed,
                               epoch_data = speed_epoch)

### Applying HsMM separately for the accuracy condition

Initializing data for the accuracy condition

In [ ]:
hmp_accuracy_data = hmp.utils.condition_selection(hmp_data, epoch_data, 'AC', variable='cue')#select the conditions where participants needs to be fast
init_accuracy = hmp.models.hmp(hmp_accuracy_data,
                       sfreq=epoch_data.sfreq, cpus=cpus)

In [ ]:
estimates_accuracy = init_accuracy.fit()

In [ ]:
hmp.visu.plot_topo_timecourse(epoch_data, estimates_accuracy, positions, init_accuracy, 
                          times_to_display = np.mean(init_accuracy.ends - init_accuracy.starts))

In [ ]:
accuracy_epoch = epoch_data.where(epoch_data['cue'].str.contains('AC'),drop=True)
bootstrapped = hmp.resample.bootstrapping(data=accuracy_epoch, dim=['epochs','participant'], n_iterations=10, 
                                                 init=init_accuracy, positions=positions, sfreq=epoch_data.sfreq, cpus=cpus)

hmp.visu.plot_bootstrap_results(bootstrapped, positions, init_accuracy,
                               epoch_data = accuracy_epoch)

In [ ]:
for condition in zip([estimates_speed, estimates_accuracy],
                     [init_speed, init_accuracy],
                     [hmp_speed_data, hmp_accuracy_data],
                    ['speed    ', 'Accuracy']):
    hmp.visu.plot_topo_timecourse(epoch_data, condition[0],
                          positions, condition[1],  
                          times_to_display = np.mean(condition[1].ends - condition[1].starts),max_time=100,
                          figsize=(10,1), ylabels={'Condition':[condition[3]]})

As well as the latencies of each stage

In [ ]:
condition_times = []
for cond in zip([init_speed, init_accuracy],
                [estimates_speed, estimates_accuracy]):#Not very neat, to replace later
    condition_times.append(cond[0].compute_times(cond[0], cond[1].dropna('event'), fill_value=0, add_rt=True))
condition_times = np.array(condition_times,dtype=object)

In [ ]:
hmp.visu.plot_latencies_average(condition_times, time_step=1000/init_accuracy.sfreq, 
                labels= ['speed', 'Accuracy'], errs='ci');

Or with a line plot

In [ ]:
condition_times = []
for cond in zip([init_speed, init_accuracy],
                [estimates_speed, estimates_accuracy]):#Not very neat, to replace later
    condition_times.append(cond[0].compute_times(cond[0], cond[1].dropna('event'), add_rt=True))
condition_times = np.array(condition_times,dtype=object)
hmp.visu.plot_latencies(condition_times, kind='point',legend=True,
    labels= ['speed', 'Accuracy'], errs='ci', time_step=1000/init_accuracy.sfreq,
    colors=["indianred","darkgreen"], figsize=(10,5));

# Putting the Event in Event Related Potentials

In [ ]:
# The following code is just to illustrate channel positions
import mne

epoch = mne.read_epochs(os.path.join('../sample_data/eeg/processed_0022_epo.fif'))
epoch.plot_sensors(show_names=True);


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

BRP_times_speed = init_speed.compute_times(init_speed, estimates_speed.dropna('event'), fill_value=0, add_rt=True)
BRP_times_accuracy = init_accuracy.compute_times(init_accuracy, estimates_accuracy.dropna('event'), fill_value=0, add_rt=True)
data_speed = speed_epoch.stack({'trial_x_participant':['participant','epochs']}).data.dropna('trial_x_participant', how="all")
data_accuracy = accuracy_epoch.stack({'trial_x_participant':['participant','epochs']}).data.dropna('trial_x_participant', how="all")
fig, ax = plt.subplots(1,5, figsize=(20,5), sharey=True)
for stage in [0,1,2,3,4]:
    if stage < 4:#As speed as 1 less stage
        BRP = hmp.utils.event_times(data_speed, BRP_times_speed,'CPz', stage=stage)
        df = pd.DataFrame(BRP).melt(var_name='Time')
        sns.lineplot(x="Time", y="value", data=df,ax=ax[stage], color='indianred')
    BRP = hmp.utils.event_times(data_accuracy, BRP_times_accuracy,'CPz',stage=stage)
    df = pd.DataFrame(BRP).melt(var_name='Time')
    sns.lineplot(x="Time", y="value", data=df,ax=ax[stage], color='darkgreen')


BRPs are hardly comparable, for the demo we can select the 3 events also in the accuracy condition

In [ ]:
estimates_accuracy_3events = init_accuracy.fit_single(n_events=3, magnitudes=estimates_accuracy.sel(event=[0,1,3]).magnitudes, magnitudes_to_fix=[0,1,2])

hmp.visu.plot_topo_timecourse(epoch_data, estimates_accuracy_3events, positions, init_accuracy, 
                          times_to_display = np.mean(init_accuracy.ends - init_accuracy.starts))

In [ ]:
BRP_times_speed = init_speed.compute_times(init_speed, estimates_speed.dropna('event'), fill_value=0, add_rt=True).dropna('event')
BRP_times_accuracy = init_accuracy.compute_times(init_accuracy, estimates_accuracy_3events.dropna('event'), fill_value=0, add_rt=True).dropna('event')
fig, ax = plt.subplots(1,4, figsize=(20,5), sharey=True, sharex=False)

time_limit_speed = np.percentile(np.diff(BRP_times_speed,axis=1), q=90, axis=0)
time_limit_accuracy = np.percentile(np.diff(BRP_times_accuracy,axis=1), q=90, axis=0)

for stage in [0,1,2,3]:
    BRP = hmp.utils.event_times(data_speed, BRP_times_speed,'CPz', stage=stage)
    df = pd.DataFrame(BRP).melt(var_name='Time')
    sns.lineplot(x="Time", y="value", data=df[df.Time < time_limit_speed[stage]],ax=ax[stage], color='indianred')
    BRP = hmp.utils.event_times(data_accuracy, BRP_times_accuracy,'CPz', stage=stage)
    df = pd.DataFrame(BRP).melt(var_name='Time')
    sns.lineplot(x="Time", y="value", data=df[df.Time < time_limit_accuracy[stage]],ax=ax[stage], color='darkgreen')

# Comparing left and right responses

We expect that most of the information should be shared across so we can fit a model on all the data and then estimate separate models

In [ ]:
init = hmp.models.hmp(hmp_data, sfreq=epoch_data.sfreq, cpus=cpus)
fit = init.fit()
hmp.visu.plot_topo_timecourse(epoch_data, fit, positions, init)

In [ ]:
hmp_l_data = hmp.utils.condition_selection(hmp_data, epoch_data, 'resp_left', variable='resp')
init_l = hmp.models.hmp(hmp_l_data, sfreq=epoch_data.sfreq, cpus=cpus)

hmp_r_data = hmp.utils.condition_selection(hmp_data, epoch_data, 'resp_right', variable='resp')
init_r = hmp.models.hmp(hmp_r_data, sfreq=epoch_data.sfreq, cpus=cpus)

In [ ]:
left_resp_estimates = init_l.fit_single(3, magnitudes=fit.magnitudes)
right_resp_estimates = init_r.fit_single(3, magnitudes=fit.magnitudes)

In [ ]:
for condition in zip([left_resp_estimates, right_resp_estimates],
                     [init_l, init_r],
                     [hmp_l_data, hmp_r_data],
                    ['Left   ', 'Right']):
    hmp.visu.plot_topo_timecourse(epoch_data, condition[0], positions, condition[1], 
                          times_to_display = np.mean(condition[1].ends - condition[1].starts),
                          max_time=95, figsize=(10,1), ylabels={'Condition':[condition[3]]})

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

BRP_times_left = init_l.compute_times(init_l, left_resp_estimates.dropna('event'), fill_value=0, add_rt=True)
BRP_times_right = init_r.compute_times(init_r, right_resp_estimates.dropna('event'), fill_value=0, add_rt=True)
data_left = epoch_data.where(epoch_data['resp'].str.contains('resp_left'), drop=True)\
        .stack({'trial_x_participant':['participant','epochs']}).data.dropna('trial_x_participant', how="all")
data_right = epoch_data.where(epoch_data['resp'].str.contains('resp_right'), drop=True)\
        .stack({'trial_x_participant':['participant','epochs']}).data.dropna('trial_x_participant', how="all")

fig, ax = plt.subplots(1,2, figsize=(8,5), sharey=True)

time_limit = np.percentile(np.diff(BRP_times_left,axis=1), q=90, axis=0)

stage = 3
BRP_contra = hmp.utils.event_times(data_left, BRP_times_left,'C4', stage=stage)
df = pd.DataFrame(BRP_contra).melt(var_name='Time')
df.value = df.value -df[df.Time==0].value.mean()
sns.lineplot(x="Time", y="value", data=df[df.Time < time_limit[stage]],ax=ax[0], color='indianred', label="contralateral")
BRP_ipsi = hmp.utils.event_times(data_left, BRP_times_left,'C3', stage=stage)
df = pd.DataFrame(BRP_ipsi).melt(var_name='Time')
df.value = df.value - df[df.Time==0].value.mean()
sns.lineplot(x="Time", y="value", data=df[df.Time < time_limit[stage]],ax=ax[0], color='darkgreen', label="ipsilateral")
ax[0].legend()

BRP_contra = hmp.utils.event_times(data_right, BRP_times_right,'C3', stage=stage)
df = pd.DataFrame(BRP_contra).melt(var_name='Time')
df.value = df.value - df[df.Time==0].value.mean()
sns.lineplot(x="Time", y="value", data=df[df.Time < time_limit[stage]],ax=ax[1], color='indianred')
BRP_ipsi = hmp.utils.event_times(data_right, BRP_times_right,'C4', stage=stage)
df = pd.DataFrame(BRP_ipsi).melt(var_name='Time')
df.value = df.value -df[df.Time==0].value.mean()
sns.lineplot(x="Time", y="value", data=df[df.Time < time_limit[stage]],ax=ax[1], color='darkgreen');
